<a href="https://colab.research.google.com/github/chrisseiler96/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
# TODO


import numpy as np



import pandas as pd



In [7]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [15]:
df[0].value_counts()

b    468
a    210
?     12
Name: 0, dtype: int64

In [0]:
df.replace('?', np.nan, inplace = True)

In [20]:
df.isnull().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

In [0]:
pos_df = df.loc[df[15] == '+']
neg_df = df.loc[df[15] == '-']

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [26]:
df.shape

(690, 16)

In [27]:
neg_df.shape

(383, 16)

In [28]:
pos_df.shape

(307, 16)

In [30]:
pos_df.describe()

,2,7,10,14
count,307.000000,307.000000,307.000000,307.000000
mean,5.904951,3.427899,4.605863,2038.859935
std,5.471485,4.120792,6.320242,7659.763941
min,0.000000,0.000000,0.000000,0.000000
25%,1.500000,0.750000,0.000000,0.000000
50%,4.460000,2.000000,3.000000,221.000000
75%,9.520000,5.000000,7.000000,1209.000000
max,28.000000,28.500000,67.000000,100000.000000


In [31]:
neg_df.describe()

,2,7,10,14
count,383.000000,383.000000,383.000000,383.000000
mean,3.839948,1.257924,0.631854,198.605744
std,4.337662,2.120481,1.900049,671.608839
min,0.000000,0.000000,0.000000,0.000000
25%,0.835000,0.125000,0.000000,0.000000
50%,2.210000,0.415000,0.000000,1.000000
75%,5.000000,1.500000,0.000000,67.000000
max,26.335000,13.875000,20.000000,5552.000000


In [43]:
pos_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [54]:
neg_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
70,b,32.33,7.5,u,g,e,bb,1.585,t,f,0,t,s,00420,0,-
71,b,34.83,4.0,u,g,d,bb,12.500,t,f,0,t,g,NaN,0,-
72,a,38.58,5.0,u,g,cc,v,13.500,t,f,0,t,g,00980,0,-
73,b,44.25,0.5,u,g,m,v,10.750,t,f,0,f,s,00400,0,-
74,b,44.83,7.0,y,p,c,v,1.625,f,f,0,f,g,00160,2,-


#TEST 1 - Pre-Processing

## A11 - CONTINUOUS 

In [0]:
test1_pos = pos_df[[10]]

In [50]:
test1_pos.head()

,10
0,1
1,6
2,0
3,5
4,0


In [0]:
test1_neg = neg_df[[10]]

In [53]:
test1_neg.head()

,10
70,0
71,0
72,0
73,0
74,0


In [55]:
test1_pos.describe()

,10
count,307.000000
mean,4.605863
std,6.320242
min,0.000000
25%,0.000000
50%,3.000000
75%,7.000000
max,67.000000


In [56]:
test1_neg.describe()

,10
count,383.000000
mean,0.631854
std,1.900049
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,20.000000


In [0]:
from scipy import stats




---

# Test 1 Hypothesis
### The Mean of Column A11 for people approved APPEARS significantly higher than the mean of people declined.

---




##$H_0$ = Column A11 was not a factor used to determine if someone was approved or denied credit

##$H_a$ = Column A11 was a factor that contributed to someone's approval for credit



In [62]:
print(stats.ttest_ind(test1_pos,test1_neg).statistic)

[11.66700422]


In [63]:
print(stats.ttest_ind(test1_pos,test1_neg).pvalue)

[7.95771857e-29]




---


### Mean for those accepted: 4.6
### Standard Deviation: 6.32


---



### Mean for those declined: 0.63
### Standard Deviation: 1.90




---

## The T Score is 11.66!

## The P Value is [7.95771857e-29] - This is an EXTREMELY small number.


---



##The probability that a difference this large was due to pure chance is so low it has to be expressed in scientific notation.


---

# I reject the Null Hypothesis. A given person's A11 Value was a factor in deciding credit approval or disapproval.

#TEST 2 - Pre-Processing

In [64]:
pos_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [65]:
neg_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
70,b,32.33,7.5,u,g,e,bb,1.585,t,f,0,t,s,00420,0,-
71,b,34.83,4.0,u,g,d,bb,12.500,t,f,0,t,g,NaN,0,-
72,a,38.58,5.0,u,g,cc,v,13.500,t,f,0,t,g,00980,0,-
73,b,44.25,0.5,u,g,m,v,10.750,t,f,0,f,s,00400,0,-
74,b,44.83,7.0,y,p,c,v,1.625,f,f,0,f,g,00160,2,-


In [0]:
test2_pos_trial = pos_df[[1]]
test2_pos_trial.head(50)

In [94]:
test2_pos_trial.isna().sum()

1    2
dtype: int64

In [0]:
test2_pos_trial = test2_pos_trial.dropna()

In [97]:
test2_pos_trial.isna().sum()

1    0
dtype: int64



---



In [0]:
test2_neg_trial = neg_df[[1]]
test2_neg_trial.head(10)

In [106]:
test2_neg_trial.isna().sum()

1    10
dtype: int64

In [0]:
test2_neg_trial = test2_neg_trial.dropna()

In [108]:
test2_neg_trial.isna().sum()

1    0
dtype: int64

In [109]:
test2_neg_trial.shape

(373, 1)

In [110]:
test2_neg_trial.describe()

,1
count,373
unique,231
top,23.58
freq,6




---



In [0]:
test2_pos = test2_pos_trial
test2_neg = test2_neg_trial

In [102]:
test2_pos.head()

,1
0,30.83
1,58.67
2,24.50
3,27.83
4,20.17


In [112]:
test2_neg.head()

,1
70,32.33
71,34.83
72,38.58
73,44.25
74,44.83


In [103]:
test2_pos.shape

(305, 1)

In [113]:
test2_neg.shape

(373, 1)

In [0]:
test2_pos[[1]] = test2_pos[[1]].astype(float)
test2_neg[[1]] = test2_neg[[1]].astype(float)




---

# Test 2 Hypothesis
### The mean of column A2 appears to be relatively similar between people accepted and people declined on the application.

---




##$H_0$ = Column A2 was not a factor used to determine if someone was approved or denied credit

##$H_a$ = Column A2 was a factor that contributed to someone's approval for credit



In [118]:
test2_pos.describe()

,1
count,305.000000
mean,33.720492
std,12.809641
min,13.750000
25%,23.170000
50%,30.500000
75%,41.330000
max,76.750000


In [119]:
test2_neg.describe()

,1
count,373.000000
mean,29.808231
std,10.919291
min,15.170000
25%,22.000000
50%,27.330000
75%,34.830000
max,80.250000


In [120]:
print(stats.ttest_ind(test2_pos,test2_neg).statistic)

[4.29221562]


In [122]:
print(stats.ttest_ind(test2_pos,test2_neg).pvalue)

[2.02766371e-05]




---

### Mean for those accepted: 33.7
### Standard Deviation: 12.8


---



### Mean for those declined: 29.8
### Standard Deviation: 10.91




---

## The T Score is 4.292!

## The P Value is [2.02766371e-05]- This is an EXTREMELY small number.


---



##The probability that a difference this large was due to pure chance is so low it has to be expressed in scientific notation.


---

# I reject the Null Hypothesis. A given person's A2 Value was a factor in deciding credit approval or disapproval.

#Test 3 Pre-Processing
##A12


---



In [123]:
pos_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [124]:
neg_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
70,b,32.33,7.5,u,g,e,bb,1.585,t,f,0,t,s,00420,0,-
71,b,34.83,4.0,u,g,d,bb,12.500,t,f,0,t,g,NaN,0,-
72,a,38.58,5.0,u,g,cc,v,13.500,t,f,0,t,g,00980,0,-
73,b,44.25,0.5,u,g,m,v,10.750,t,f,0,f,s,00400,0,-
74,b,44.83,7.0,y,p,c,v,1.625,f,f,0,f,g,00160,2,-


In [0]:
test3_pos_trial = pos_df[[11]]
test3_neg_trial = neg_df[[11]]

In [127]:
test3_pos_trial.head()

,11
0,f
1,f
2,f
3,t
4,f


In [128]:
test3_neg_trial.head()

,11
70,t
71,t
72,t
73,f
74,f


In [130]:
test3_pos_trial.replace('f', 0, inplace = True)
test3_pos_trial.replace('t', 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [131]:
test3_pos_trial.head()

,11
0,0
1,0
2,0
3,1
4,0


In [132]:
test3_neg_trial.replace('f', 0, inplace = True)
test3_neg_trial.replace('t', 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [133]:
test3_neg_trial.head()

,11
70,1
71,1
72,1
73,0
74,0


In [0]:
test3_pos = test3_pos_trial
test3_neg = test3_neg_trial

In [135]:
test3_pos[[11]] = test3_pos[[11]].astype(float)
test3_neg[[11]] = test3_neg[[11]].astype(float)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [136]:
test3_pos.head()

,11
0,0.0
1,0.0
2,0.0
3,1.0
4,0.0


In [137]:
test3_neg.head()

,11
70,1.0
71,1.0
72,1.0
73,0.0
74,0.0


In [138]:
test3_pos.describe()

,11
count,307.000000
mean,0.475570
std,0.500218
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [139]:
test3_neg.describe()

,11
count,383.000000
mean,0.443864
std,0.497489
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [143]:
test3_pos.rename(columns={11: 'A12'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [144]:
test3_pos.head()

,A12
0,0.0
1,0.0
2,0.0
3,1.0
4,0.0


In [145]:
test3_neg.rename(columns={11: 'A12'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [146]:
test3_neg.head()

,A12
70,1.0
71,1.0
72,1.0
73,0.0
74,0.0


# Test 3 Hypothesis
### The mean of column A12 appears to be relatively similar between people accepted and people declined on the application.

---




##$H_0$ = Column A12 was not a factor used to determine if someone was approved or denied credit

##$H_a$ = Column A12 was a factor that contributed to someone's approval for credit



In [0]:
from scipy.stats import chisquare  # One-way chi square test

In [153]:
pd.crosstab(df[11],df[15])

15,+,-
11,,
f,161,213
t,146,170


In [0]:
test3_xtab = pd.crosstab(df[11],df[15])

In [155]:
print(chisquare(test3_xtab, axis=None))

Power_divergenceResult(statistic=14.382608695652173, pvalue=0.0024280203352694887)


In [0]:
test3_xtab.replace('f', 0, inplace = True)
test3_xtab.replace('t', 1, inplace = True)

In [157]:
print(chisquare(test3_xtab, axis=None))

Power_divergenceResult(statistic=14.382608695652173, pvalue=0.0024280203352694887)


In [158]:
test3_xtab.head()

15,+,-
11,,
f,161,213
t,146,170




---

#The p-value is very low - the likelyhood of seeing a result this extreme or more extreme is very low.
## I reject the null hypothesis, there is a chi square of 14


---



#Test 4 Pre-Processing
##A5


---


In [0]:
clean_df = df.dropna()

In [161]:
clean_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [0]:
test4_xtab = pd.crosstab(df[4],df[15])

In [163]:
test4_xtab.head()

15,+,-
4,,
g,256,263
gg,2,0
p,45,118


# Test 4 Hypothesis
### column A5

---




##$H_0$ = Column A5 was not a factor used to determine if someone was approved or denied credit

##$H_a$ = Column A5 was a factor that contributed to someone's approval for credit


In [164]:
print(chisquare(test4_xtab, axis=None))

Power_divergenceResult(statistic=637.561403508772, pvalue=1.545055409454076e-135)


#There is a very high test statistic and a very small P value  - this means we can reject the null hypothesis, it's very unlikely this was due to chance.


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.



---


###I explained the relationships in part 2 along with the actual tests, oddly enough it seemed like every attribute on the credit application played a significant role in deciding whether a person was accepted or declined... I guess that makes sense. I'm sure within the data there exists an opposite case but I'm writing this with about 5 minutes left to submit this, so I don't have time to dig deeper.

###This sprint, and this entire week was very challenging for me because I don't have a statistics background. To be honest, I'm still not 100% sure why running a Chi Square requires crosstabs... trying to run the Chi Square tests was the most challenging part for me. 